In [1]:
#importando pacotes

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
import numpy as np


In [10]:
df_countries = pd.read_csv("database/best-countries-to-live-in-2024.csv", encoding='ISO-8859-1') #branch da madu
df_country_statistics = pd.read_csv("database/Country statistics.csv", encoding='ISO-8859-1')  #branch do daniel
df_data = pd.read_csv("database/data.csv", encoding='ISO-8859-1') #gigante, vamos ver ela por último em grupo
df_men = pd.read_csv("database/men.csv", encoding='ISO-8859-1') #branch da maya
df_ufc_fighters = pd.read_csv("database/ufc-fighters-statistics.csv", encoding='ISO-8859-1') #branch do pedro
df_idh = pd.read_csv("database/human-development-index.csv", encoding='ISO-8859-1')
df_idh.head()


,Entity,Code,Year,Human Development Index
0,Afghanistan,AFG,1990,0.284
1,Afghanistan,AFG,1991,0.292
2,Afghanistan,AFG,1992,0.299
3,Afghanistan,AFG,1993,0.307
4,Afghanistan,AFG,1994,0.300


In [11]:
#verificando o conteúdo do dataset
df_countries.info()


countries_list = df_countries.country.unique().tolist() 
idh_list = df_idh.Entity.unique().tolist()

#verificando quais países do dataset countries não estão no dataset de idh
countries_not_in_list = []
for pais in idh_list:
    if(pais not in countries_list):
        countries_not_in_list.append(pais)

print(countries_not_in_list)

#limpeza do dataset best countries to live
new_df_countries = df_countries.drop(columns=['population_2024', 'population_growthRate', 'land_area', 'population_density', 'population_densityMi', 'share_borders'])

#verificando se a valores nulos (nan) no dataset
nan_in_df = new_df_countries.isnull().sum().sum()
 
# printing the number of values present in
# the whole dataframe
print('Number of NaN values present: ' + str(nan_in_df) + "\n")


new_df_countries



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   population_2024        141 non-null    int64  
 1   population_growthRate  141 non-null    float64
 2   land_area              141 non-null    int64  
 3   country                141 non-null    object 
 4   region                 141 non-null    object 
 5   unMember               141 non-null    bool   
 6   population_density     141 non-null    float64
 7   population_densityMi   141 non-null    float64
 8   share_borders          129 non-null    object 
 9   Hdi2021                141 non-null    float64
 10  Hdi2020                141 non-null    float64
 11  WorldHappiness2022     141 non-null    float64
dtypes: bool(1), float64(6), int64(2), object(3)
memory usage: 12.4+ KB
['Andorra', 'Angola', 'Antigua and Barbuda', 'Arab States (UNDP)', 'Bahamas', 'Barbados', 'Beli

,country,region,unMember,Hdi2021,Hdi2020,WorldHappiness2022
0,India,Asia,True,0.633,0.642,3.777
1,China,Asia,True,0.768,0.764,5.585
2,United States,North America,True,0.921,0.920,6.977
3,Indonesia,Asia,True,0.705,0.709,5.240
4,Pakistan,Asia,True,0.544,0.543,4.516
...,...,...,...,...,...,...
136,Comoros,Africa,True,0.558,0.562,4.609
137,Luxembourg,Europe,True,0.930,0.924,7.404
138,Montenegro,Europe,True,0.832,0.826,5.547
139,Malta,Europe,True,0.918,0.911,6.447


Verificado que o dataset de best countries to live não possui entradas nulas \o/
As colunas que fora dropadas são aquelas que não possuem objetivo relacionado ao nosso trabalho (podem ocorrer mudanças é claro).
Foi verificado que alguns países não estão inseridos no dataset de best countries to live, mas não cabe nessa instância retirar esses valores da tabela ou colocá-los, pois uma melhor métrica seria saber quais são os países de cada lutador da base de dados, e então fazer um filtro



Parte do Daniel:

Dataset original = 6171 linhas e 4 colunas

Retirei as linhas que tinham valores nulos da coluna "CODE" = 5841 linhas

Após isso deixei apenas os dados que são a partir do ano de 1997 = 4818

Perda de 1.353 linhas




In [46]:
#dataset human-devolopment-index
df_idh.head(10)

,Entity,Code,Year,Human Development Index
0,Afghanistan,AFG,1990,0.284
1,Afghanistan,AFG,1991,0.292
2,Afghanistan,AFG,1992,0.299
3,Afghanistan,AFG,1993,0.307
4,Afghanistan,AFG,1994,0.300
5,Afghanistan,AFG,1995,0.318
6,Afghanistan,AFG,1996,0.326
7,Afghanistan,AFG,1997,0.330
8,Afghanistan,AFG,1998,0.329
9,Afghanistan,AFG,1999,0.337


In [47]:
df_idh.shape

(6171, 4)

In [49]:
valores_nulos = df_idh.isnull().sum()
print(valores_nulos)

Entity                       0
Code                       330
Year                         0
Human Development Index      0
dtype: int64


In [50]:
linhas_com_nulos = df_idh[df_idh.isnull().any(axis=1)]
print(linhas_com_nulos)

                                  Entity Code  Year  Human Development Index
162                   Arab States (UNDP)  NaN  1990                    0.550
163                   Arab States (UNDP)  NaN  1991                    0.555
164                   Arab States (UNDP)  NaN  1992                    0.561
165                   Arab States (UNDP)  NaN  1993                    0.570
166                   Arab States (UNDP)  NaN  1994                    0.579
...                                  ...  ...   ...                      ...
6001  Very high human development (UNDP)  NaN  2018                    0.898
6002  Very high human development (UNDP)  NaN  2019                    0.901
6003  Very high human development (UNDP)  NaN  2020                    0.895
6004  Very high human development (UNDP)  NaN  2021                    0.896
6005  Very high human development (UNDP)  NaN  2022                    0.902

[330 rows x 4 columns]


In [51]:
pd.set_option('display.max_rows', None)
# Exibindo todas as linhas com valores nulos
print(linhas_com_nulos)

                                      Entity Code  Year  \
162                       Arab States (UNDP)  NaN  1990   
163                       Arab States (UNDP)  NaN  1991   
164                       Arab States (UNDP)  NaN  1992   
165                       Arab States (UNDP)  NaN  1993   
166                       Arab States (UNDP)  NaN  1994   
167                       Arab States (UNDP)  NaN  1995   
168                       Arab States (UNDP)  NaN  1996   
169                       Arab States (UNDP)  NaN  1997   
170                       Arab States (UNDP)  NaN  1998   
171                       Arab States (UNDP)  NaN  1999   
172                       Arab States (UNDP)  NaN  2000   
173                       Arab States (UNDP)  NaN  2001   
174                       Arab States (UNDP)  NaN  2002   
175                       Arab States (UNDP)  NaN  2003   
176                       Arab States (UNDP)  NaN  2004   
177                       Arab States (UNDP)  NaN  2005 

In [31]:
# Removendo todas as linhas com valores nulos do DataFrame original
df_idh.dropna(inplace=True)
# Verificando se as linhas com valores nulos foram removidas
print(df_idh)


           Entity Code  Year  Human Development Index
0     Afghanistan  AFG  1990                    0.284
1     Afghanistan  AFG  1991                    0.292
2     Afghanistan  AFG  1992                    0.299
3     Afghanistan  AFG  1993                    0.307
4     Afghanistan  AFG  1994                    0.300
...           ...  ...   ...                      ...
6166     Zimbabwe  ZWE  2018                    0.564
6167     Zimbabwe  ZWE  2019                    0.560
6168     Zimbabwe  ZWE  2020                    0.554
6169     Zimbabwe  ZWE  2021                    0.549
6170     Zimbabwe  ZWE  2022                    0.550

[5841 rows x 4 columns]


In [53]:
df_idh.shape

(5841, 4)

#limpeza feita pelo marcelo

In [17]:
df_pro_mma = pd.read_csv('database/pro_mma_fighters.csv', encoding='ISO-8859-1') #branch do marcelo 
# Verificar valores nulos em cada coluna
null_values = df_pro_mma.isnull().sum()

# Exibir colunas com valores nulos e suas respectivas quantidades
print(null_values)

df_pro_mma.head()

url                     0
fighter_name            3
nickname             1946
birth_date           1115
age                  1149
death_date           5114
location              512
country                22
height                  0
weight                  0
association           727
weight_class           30
wins                    0
wins_ko                 0
wins_submission         0
wins_decision           0
wins_other           4722
lossess                 0
losses_ko               0
losses_submission       0
losses_decision         0
losses_other         4722
dtype: int64


,url,fighter_name,nickname,birth_date,age,death_date,location,country,height,weight,...,wins,wins_ko,wins_submission,wins_decision,wins_other,lossess,losses_ko,losses_submission,losses_decision,losses_other
0,/fighter/Kharun-Atlangeriev-167431,Kharun Atlangeriev,Predator,05/10/1992,28.0,NaN,NaN,Russia,"5'10""",155 lbs,...,11,5,5,1,NaN,11,0,1,1,NaN
1,/fighter/Shojin-Miki-157565,Shojin Miki,Mini Margarito,NaN,NaN,NaN,"Honolulu, Hawaii",United States,"5'9""",125 lbs,...,6,2,4,0,NaN,6,2,0,5,NaN
2,/fighter/Ryan-Keenan-33827,Ryan Keenan,Who Dat?,15/06/1985,36.0,NaN,"Windermere, Florida",United States,"6'0""",155 lbs,...,6,3,3,0,NaN,6,1,1,0,NaN
3,/fighter/Dustin-Jacoby-73825,Dustin Jacoby,The Hanyak,04/04/1988,33.0,NaN,"Arenzville, Illinois",United States,"6'4""",205 lbs,...,14,9,1,4,NaN,14,1,2,2,NaN
4,/fighter/Richardson-Moreira-72825,Richardson Moreira,Rick Monstro,31/03/1984,37.0,NaN,"Campinas, Sao Paulo",Brazil,"6'2""",205 lbs,...,8,2,5,1,NaN,8,1,1,3,NaN


In [19]:
# Função para atribuir a classe de peso com base no peso
def definir_classe_peso(weight):
    weight = int(weight.split()[0])  # Pega apenas o número do peso
    if weight <= 115:
        return 'Strawweight'
    elif weight <= 125:
        return 'Flyweight'
    elif weight <= 135:
        return 'Bantamweight'
    elif weight <= 145:
        return 'Featherweight'
    elif weight <= 155:
        return 'Lightweight'
    elif weight <= 170:
        return 'Welterweight'
    elif weight <= 185:
        return 'Middleweight'
    elif weight <= 205:
        return 'Light Heavyweight'
    elif weight <= 265:
        return 'Heavyweight'
    else:
        return 'Super Heavyweight'

# Preencher valores nulos em weight_class com base em weight
df_pro_mma['weight_class'] = df_pro_mma['weight_class'].fillna(df_pro_mma['weight'].apply(definir_classe_peso))

# Verificar se há mais valores nulos na coluna weight_class
print(df_pro_mma['weight_class'].isnull().sum())



0


In [20]:
# Excluir as linhas que possuem valores nulos nas colunas 'fighter_name' ou 'country'
df_pro_mma_cleaned = df_pro_mma.dropna(subset=['fighter_name', 'country'])

# Verificar se as linhas com valores nulos nessas colunas foram removidas
print(df_pro_mma_cleaned.isnull().sum())

url                     0
fighter_name            0
nickname             1922
birth_date           1094
age                  1128
death_date           5089
location              490
country                 0
height                  0
weight                  0
association           706
weight_class            0
wins                    0
wins_ko                 0
wins_submission         0
wins_decision           0
wins_other           4700
lossess                 0
losses_ko               0
losses_submission       0
losses_decision         0
losses_other         4700
dtype: int64


A partir daqui começa o filtro das tabelas de IDH e best-coutries-to-live. Os páises da tabela vão ficar restritos as nacionalidades dos lutadores 

In [21]:
df_pro_mma_cleaned = df_pro_mma_cleaned[~df_pro_mma_cleaned["country"].isin(("INDIA", "Unknown"))]
df_pro_mma_cleaned = df_pro_mma_cleaned.dropna(how='any',subset=["country"])
lista_nacionalidades = df_pro_mma_cleaned.country.unique().tolist()
print(len(lista_nacionalidades))
print(lista_nacionalidades)

124
['Russia', 'United States', 'Brazil', 'Australia', 'France', 'Papua New Guinea', 'China', 'Japan', 'Italy', 'Malaysia', 'Moldova', 'Czech Republic', 'Philippines', 'Egypt', 'Denmark', 'Scotland', 'Israel', 'Canada', 'South Korea', 'Cambodia', 'Argentina', 'USA', 'England', 'Ireland', 'Myanmar', 'Indonesia', 'Iran', 'Kazakhstan', 'Mexico', 'Bulgaria', 'Finland', 'Poland', 'Liberia', 'Congo', 'Portugal', 'Chile', 'Ukraine', 'Austria', 'Switzerland', 'South Africa', 'Thailand', 'Germany', 'Sweden', 'Colombia', 'Costa Rica', 'Lithuania', 'Nigeria', 'New Zealand', 'Peru', 'Uruguay', 'Netherlands', 'Cameroon', 'Aruba', 'Slovakia', 'India', 'Singapore', 'Kyrgyzstan', 'Croatia', 'Panama', 'Azerbaijan', 'Tunisia', 'Spain', 'Cyprus', 'Belgium', 'Pakistan', 'Jamaica', 'Belarus', 'Bosnia and Herzegovina', 'Mongolia', 'Congo, The Democratic Republic of the', 'Romania', 'Venezuela', 'Georgia', 'Turkmenistan', 'Northern Ireland', 'Norway', 'Turkey', 'Suriname', 'Greece', 'Taiwan', 'Estonia', 'Lat

In [40]:
#filtrando as tabelas de idh e best countries
new_df_countries = new_df_countries[new_df_countries["country"].isin(lista_nacionalidades)]
df_idh = df_idh[df_idh["Entity"].isin(lista_nacionalidades)]

#também vamos limitar o intervalo de anos de idh - a partir de 1997
df_idh = df_idh[df_idh["Year"] >= 1997]

print("dimensão do dataset de best-countries pós filtro: " + str(new_df_countries.shape))
print("\ndimensão do dataset de idh pós filtro: " + str(df_idh.shape))

df_idh.head()

dimensão do dataset de best-countries pós filtro: (96, 6)

dimensão do dataset de idh pós filtro: (2775, 4)


,Entity,Code,Year,Human Development Index
7,Afghanistan,AFG,1997,0.330
8,Afghanistan,AFG,1998,0.329
9,Afghanistan,AFG,1999,0.337
10,Afghanistan,AFG,2000,0.340
11,Afghanistan,AFG,2001,0.344


In [63]:
l_idh = df_idh.Entity.unique().tolist()
l_countries = new_df_countries.country.unique().tolist()

for country in lista_nacionalidades:
    if(country not in l_idh):
        print(country + "\n")

Czech Republic

Scotland

USA

England

Aruba

Congo, The Democratic Republic of the

Northern Ireland

Taiwan

Wales

Guam

United states

Northern Mariana Islands

Macedonia

Puerto Rico

Brunei Darussalam



In [64]:
#tentando concatenar todas as informações sobre os países
countries_region_list = []
for linha in new_df_countries.itertuples():
    countries_region_list.append([linha.country, linha.region])

print(countries_region_list)

[['India', 'Asia'], ['China', 'Asia'], ['United States', 'North America'], ['Indonesia', 'Asia'], ['Pakistan', 'Asia'], ['Nigeria', 'Africa'], ['Brazil', 'South America'], ['Bangladesh', 'Asia'], ['Russia', 'Europe'], ['Mexico', 'North America'], ['Japan', 'Asia'], ['Philippines', 'Asia'], ['Egypt', 'Africa'], ['Vietnam', 'Asia'], ['Turkey', 'Asia'], ['Germany', 'Europe'], ['Thailand', 'Asia'], ['France', 'Europe'], ['South Africa', 'Africa'], ['Italy', 'Europe'], ['Myanmar', 'Asia'], ['Colombia', 'South America'], ['South Korea', 'Asia'], ['Spain', 'Europe'], ['Iraq', 'Asia'], ['Argentina', 'South America'], ['Afghanistan', 'Asia'], ['Poland', 'Europe'], ['Canada', 'North America'], ['Morocco', 'Africa'], ['Ukraine', 'Europe'], ['Uzbekistan', 'Asia'], ['Peru', 'South America'], ['Malaysia', 'Asia'], ['Nepal', 'Asia'], ['Cameroon', 'Africa'], ['Australia', 'Oceania'], ['Kazakhstan', 'Asia'], ['Chile', 'South America'], ['Romania', 'Europe'], ['Ecuador', 'South America'], ['Senegal', 'A

In [67]:
region_column = []
qtd_linhas = 0
for linha in df_idh.itertuples():
    qtd_linhas+=1
    for countrie in countries_region_list:
        if countrie[0] == linha.Entity:
            region_column.append(countrie[1])

print(qtd_linhas)
print(len(region_column))
print(region_column)



2775
2431
['Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'Europe', 'South America', 'South America', 'South America', 'South America', 'South America', 'South America', 'South America', 'South America', 'South America', 'South America', 'South America', 'South America', 'South America', 'South America', 'South America', 'South America', 'South America', 'South America', 'South America', 'South America', 'South America', 'South America', 'South America', 'South America', 'South America', 'South America', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia', 'Asia',

In [45]:
df_region = pd.DataFrame({
    "Region": region_column
})

new_idh_df = pd.concat([df_idh, df_region], axis=1)

new_idh_df

ValueError: Length of values (2431) does not match length of index (2775)